# Import libraries

In [1]:
import sys
import os
sys.path.append("..")  # Adds higher directory to python modules path.
#from img_to_vec import Img2Vec
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from skimage import io
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QInputDialog, QLineEdit, QFileDialog
from matplotlib import pyplot as plt
import timeit
#import cv2
#from cv2_rolling_ball import subtract_background_rolling_ball

# User Input

In [2]:
import sys
from PyQt5 import QtCore, QtWidgets
from PyQt5.QtWidgets import QApplication, QWidget, QFileDialog

app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication(sys.argv)
input_path = QFileDialog.getExistingDirectory(caption = 'Select files')
print("Active directory: ",input_path)

Active directory:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj


In [3]:
File_filter = ".tif"
Auto_write = True
Run_with_cuda = True

# Guts

In [4]:
#Img2Vec from GitHub, https://github.com/christiansafka/img2vec

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

class Img2Vec():

    def __init__(self, cuda=False, model='resnet-18', layer='default', layer_output_size=512):
        """ Img2Vec
        :param cuda: If set to True, will run forward pass on GPU
        :param model: String name of requested model
        :param layer: String or Int depending on model.  See more docs: https://github.com/christiansafka/img2vec.git
        :param layer_output_size: Int depicting the output size of the requested layer
        """
        self.device = torch.device("cuda" if cuda else "cpu")
        self.layer_output_size = layer_output_size
        self.model_name = model
        
        self.model, self.extraction_layer = self._get_model_and_layer(model, layer)

        self.model = self.model.to(self.device)

        self.model.eval()

        self.scaler = transforms.Resize((224, 224))
        self.RandomRotation = transforms.RandomRotation(degrees=90) #Added rotational invariance for this application
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        self.to_tensor = transforms.ToTensor()

    def get_vec(self, img, tensor=False):
        """ Get vector embedding from PIL image
        :param img: PIL Image
        :param tensor: If True, get_vec will return a FloatTensor instead of Numpy array
        :returns: Numpy ndarray
        """
        image = self.normalize(self.to_tensor(self.scaler(img))).unsqueeze(0).to(self.device)

        if self.model_name == 'alexnet':
            my_embedding = torch.zeros(1, self.layer_output_size)
        else:
            my_embedding = torch.zeros(1, self.layer_output_size, 1, 1)

        def copy_data(m, i, o):
            my_embedding.copy_(o.data)

        h = self.extraction_layer.register_forward_hook(copy_data)
        h_x = self.model(image)
        h.remove()

        if tensor:
            return my_embedding
        else:
            if self.model_name == 'alexnet':
                return my_embedding.numpy()[0, :]
            else:
                return my_embedding.numpy()[0, :, 0, 0]

    def _get_model_and_layer(self, model_name, layer):
        """ Internal method for getting layer from model
        :param model_name: model name such as 'resnet-18'
        :param layer: layer as a string for resnet-18 or int for alexnet
        :returns: pytorch model, selected layer
        """
        if model_name == 'resnet-18':
            model = models.resnet18(pretrained=True)
            if layer == 'default':
                layer = model._modules.get('avgpool')
                self.layer_output_size = 512
            else:
                layer = model._modules.get(layer)

            return model, layer

        elif model_name == 'alexnet':
            model = models.alexnet(pretrained=True)
            if layer == 'default':
                layer = model.classifier[-2]
                self.layer_output_size = 4096
            else:
                layer = model.classifier[-layer]

            return model, layer

        else:
            raise KeyError('Model %s was not found' % model_name)
            

In [5]:
#Set Self
img2vec = Img2Vec(cuda=Run_with_cuda, model='resnet-18', layer='default', layer_output_size=2048)

In [6]:
start_time_all = timeit.default_timer()

pics = {}
count = 0
for subdir, dirs, files in os.walk(input_path):
    for file in files:
        start_time = timeit.default_timer()
        filepath = os.path.join(subdir, file)
        
        if filepath.endswith(File_filter) and not 'thumb' in filepath.lower():
            count = count + 1
            #Open images ending with filter
            filename = os.fsdecode(file)
            img = io.imread(filepath)
            #img, background = subtract_background_rolling_ball(img, 100, light_background=False, use_paraboloid=True, do_presmooth=True)
            
            #linearly Converts 16-bit images to 3 chan 8-bit, img2vec error w/ 16-bit input
            nimg = np.uint8(np.float32(img)/65535*255)
            arr = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
            arr[:,:,0], arr[:,:,1], arr[:,:,2] = nimg, nimg, nimg
            out_img = Image.fromarray(arr, 'RGB')
            
            #Performs neural embedding, Image -> Vector
            vec = img2vec.get_vec(out_img)
            pics[filename] = vec
            elapsed = timeit.default_timer() - start_time
            print("Finished: ",filepath,"     ",elapsed)
        
    if Auto_write:
        os.chdir(subdir)
        df = pd.DataFrame(pics)
        df = df.T
        df.index.name ='File_Name'
        if len(df)>0:
            df.to_csv('DeepHTS.csv')
            pics = {}
            
elapsed_all = timeit.default_timer() - start_time_all
print("Finished analyzing ", count, ' images. The total runtime was ',elapsed_all/60,' minutes')

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_1_t1.tif       2.6455145
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_2_t1.tif       0.6039423999999998
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_3_t1.tif       0.4515184000000003
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_4_t1.tif       0.6884783000000003
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_5_t1.tif       0.6097216999999997
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_6_t1.tif       0.5652866000000003
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_7_t1.tif       0.48093310000000056
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch01_A_8_t1.tif       0.8000912000000007
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_D_4_t1.tif       0.4024663000000004
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_D_5_t1.tif       0.4043988000000027
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_D_6_t1.tif       0.4821459000000061
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_D_7_t1.tif       0.565402000000006
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_D_8_t1.tif       0.4432389000000043
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_D_9_t1.tif       0.4263720999999947
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_E_1_t1.tif       0.3843703000000005
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020-10-13\ch02_E_2_t1.tif       0.31670890000000185
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201013-201003-no-barcode\ch02_B_4_t1.tif       0.4728227999999959
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201013-201003-no-barcode\ch02_B_5_t1.tif       0.45196070000000077
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201013-201003-no-barcode\ch02_B_6_t1.tif       0.4793427000000037
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201013-201003-no-barcode\ch02_B_7_t1.tif       0.4436496999999946
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201013-201003-no-barcode\ch02_B_8_t1.tif       0.3565322999999978
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201013-201003-no-barcode\ch02_B_9_t1.tif       0.5927724999999953
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201013-201003-no-barcode\ch02_C_1_t1.tif       0.3762275999999929
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\202

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201014-201008-no-barcode\ch01_D_7_t1.tif       0.46780540000000315
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201014-201008-no-barcode\ch01_D_8_t1.tif       0.45520889999998815
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201014-201008-no-barcode\ch01_D_9_t1.tif       0.4469916999999981
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201014-201008-no-barcode\ch01_E_1_t1.tif       0.4278951999999947
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201014-201008-no-barcode\ch01_E_2_t1.tif       0.42807470000001047
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201014-201008-no-barcode\ch01_E_3_t1.tif       0.4491270000000043
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201014-201008-no-barcode\ch01_E_4_t1.tif       0.40562899999999047
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch01_B_6_t1.tif       0.4088318000000015
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch01_B_7_t1.tif       0.4295939999999945
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch01_B_8_t1.tif       0.45242700000000013
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch01_B_9_t1.tif       0.38846089999999833
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch01_C_1_t1.tif       0.4064841000000001
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch01_C_2_t1.tif       0.4040324000000055
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch01_C_3_t1.tif       0.4443485999999979
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch02_D_9_t1.tif       0.5313195000000235
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch02_E_1_t1.tif       0.43398469999999634
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch02_E_2_t1.tif       0.6980379000000028
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch02_E_3_t1.tif       0.4661055999999917
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201015-201011-no-barcode\ch02_E_4_t1.tif       0.5929040999999984
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch01_A_1_t1.tif       0.47609919999999306
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch01_A_2_t1.tif       0.47343680000000177
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch02_B_8_t1.tif       0.6663577999999859
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch02_B_9_t1.tif       0.335491200000007
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch02_C_1_t1.tif       0.4377431000000058
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch02_C_2_t1.tif       0.3729433999999969
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch02_C_3_t1.tif       0.3245718000000011
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch02_C_4_t1.tif       0.40323239999997895
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201016-201005-no-barcode\ch02_C_5_t1.tif       0.3879794000000061
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2020

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201017-201006-no-barcode\ch01_E_2_t1.tif       0.2980636000000061
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201017-201006-no-barcode\ch01_E_3_t1.tif       0.36970039999999926
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201017-201006-no-barcode\ch01_E_4_t1.tif       0.38372209999999995
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201017-201006-no-barcode\ch02_A_1_t1.tif       0.3255582999999831
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201017-201006-no-barcode\ch02_A_2_t1.tif       0.31491630000002147
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201017-201006-no-barcode\ch02_A_3_t1.tif       0.3490111999999783
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201017-201006-no-barcode\ch02_A_4_t1.tif       0.5007659999999987
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch01_C_1_t1.tif       0.39998890000001097
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch01_C_2_t1.tif       0.6229547000000082
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch01_C_3_t1.tif       0.46363759999999843
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch01_C_4_t1.tif       0.44298079999998663
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch01_C_5_t1.tif       0.4109344999999962
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch01_C_6_t1.tif       0.3921921000000168
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch01_C_7_t1.tif       0.4434221000000207
Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\2

Finished:  U:/MX ImageStore/MDA-Nikhil/20201013-growth test_Proj\20201018-201008-no-barcode\ch02_E_4_t1.tif       0.32206589999998414
Finished analyzing  560  images. The total runtime was  4.463314698333333  minutes


# Write data

In [21]:
df = pd.DataFrame(pics)
df = df.T
df.shape
df.head()

""


In [22]:
os.getcwd()

'C:\\Users\\rpowell\\Desktop\\test'

In [ ]:
os.chdir(input_path)
df.to_csv('data.csv')
os.getcwd()

# Scratch space